# Übung 2 - Polygonzug

Folgend wird versucht, ein Polygonzug zu berechnen und mit Winkel- und Koordinatenabschluss zu verbessern. 

In [540]:
# ignore warnings
import warnings
warnings.simplefilter(action='ignore')
# Libraries
import numpy as np
import pandas as pd

In [541]:
#import data
data_Noise = pd.read_csv("./data/data_Noise.txt", delimiter=";")
data_NoNoise = pd.read_csv("./data/data_NoNoise.txt", delimiter=";")
fixpunkte = pd.read_csv("./data/fixpunkte.txt", delimiter=";")
i_meteo = pd.read_csv("./data/i_meteo.txt", delimiter=";")

# const for calcs
T = 12 #°C
P = 1013.25 #hPa
R = 6371000 #m

# "global" funcs
def radian(gons): return (gons * 2 * np.pi) / 400
def gon(rads): return (rads * 400) / (2 * np.pi)

data_NoNoise

,Visur_von,Visur_nach,RI,RII,ZI,ZII,D
0,A,P,165.5718,365.5718,103.4797,296.5203,NaN
1,A,PP1,365.7370,165.7370,96.7342,303.2658,27.4199
2,PP1,A,261.8157,61.8157,103.2658,296.7342,27.4199
3,PP1,PP2,60.9770,260.9770,97.2136,302.7864,25.4368
4,PP2,PP1,70.4558,270.4558,102.7864,297.2136,25.4368
5,PP2,PP3,251.6220,51.6220,98.3621,301.6379,33.4681
6,PP3,PP2,371.0396,171.0396,101.6379,298.3621,33.4681
7,PP3,PP4,151.6620,351.6620,99.3118,300.6882,23.3132
8,PP4,PP3,340.9986,140.9986,100.6882,299.3118,23.3132
9,PP4,E,168.9636,368.9636,100.4986,299.5014,44.8211


## Messdatenaufbereitung
Im folgenden Codeblock wird eine Funktion definiert, die als Argument pandas DataFrames vom Polygonzug selber, den gemessenen Instrumentenhöhen und Meteodaten und den verwendeten Fixpunkten einige vorbereitenden Aufbereitungen durchführt.

### Winkel
Die Horizontal- und Zenitdistanzen aus zwei Lagen werden transformiert und gemittelt.

#### Interpretation Winkel
So lange die Richtungen in beiden Lagen nahe beieinander sind (mit modernen Instrumenten <1 mgon), kann von einer guten Messungen ausgegangen werden. Im Code könnte man hier Abweichungshandling implementieren, so dass gewarnt wird, falls eine zu hohe Abweichung auftritt. Auf diese Implementierung wurde verzichtet, da dies bei modernen Geräten bei der Messung schon gewarnt wird. Dies kann durch grobe Fehler beim Bedienen (falsches Ziel), falsches Anvisieren durch die ATR (Spiegelungen des Ziels in Fensterfronten) oder bei Messfehlern geschehen (Instrumentenprüfung durchführen!).

### Informationen über Standpunkt
In der folgenden Funktion wird der DataFrame mit Informationen über die Standpunkte erweitert (Druck, Temperatur, Instrumentenhöhe).

In [542]:
# 1 Messdatenaufbereitung
def data_preprocessing(df, df_met, df_fix):
    def az_turn(az): # subtracts 200 gon from azimut
        return az - 200
        if az <= 200: ## unnötig
            return az + 200
        else: return az - 200

    # add columns with info about Standpunkt meteo
    array = pd.DataFrame()
    ## make df for every standpunkt and extract meteo info
    for i in range(len(df["Visur_von"])):
        array = pd.concat([array, df_met[df_met["PNr"] == df["Visur_von"][i]]], axis=0)
    ## concat to main df 
    df = pd.concat([df, array.reset_index()], axis=1).drop(["index", "PNr"], axis=1)
    
    ## Azimuth and zenith angles average
    #data_Noise["R_m_t"] = (data_Noise["RI"]+ data_Noise["RII"] - 200) / 2
    df["R_m"] = (df["RI"] + df["RII"].apply(az_turn)) / 2
    df["Z_m"] = (df["ZI"] + 400 - df["ZII"]) / 2

    # init h_a / h_b
    df["h_a"] = 0
    df["h_b"] = 0
    for index, punkt in df_fix.iterrows():
        df["h_a"][df["Visur_von"] == punkt["PNr"]] = float(punkt["H"]) #höhe punkt
    df["h_a"] = df["h_a"] + df["i"] #instrumentenhöhe dazu wo bekannt
    
    # Sanity check (should be small, at max 0.1 otherwise somebody f*cked up)
    #print(df["RI"] - df["RII"].apply(az_add_or_subtract))
    return df


### Aufbereitung Distanzmessungen
Im folgenden Codeblock werden Hilfsfunktionen für die Distanzreduktion und -korrektur definiert.

#### Interpretation Code Distanzen
##### Meteokorrektur
Hier hängt die Grösse der Korrektur von der Differenz der Einstellung am Gerät (oder Standard) und der effektiv gemessenen Temperatur ab. Je grösser diese ist, desto grösser ist auch die Korrektur. Weiter muss darauf geachtet werden, dass über den Tag die Temperatur konstant bleibt (nicht zu früh / zu spät messen) oder dies in der Berechnung berücksichtigt wird.

##### Sehne am Ellipsoid
Der Lichtweg im Raum kann nach der Meteokorrektur als **Sehne im Raum** (*s_0*) angenährt werden. Diese muss nun zur **Sehne am Ellipsoid** reduziert werden. Diese Reduktion hängt von dem Radius am Standpunkt (hier genähert mit konstantem Erdradius) und der Höhe der Instrumente an beiden Standpunkten. Für kurze Visuren bis 100m kann eine vereinfacht geometrisch die Höhe des Zielpunktes angenähert werden. 

##### Distanz in der Ebenen Abbildung
Da die Krümmung des Ellipsoids bei kurzen Distanzen vernächlässigbar ist, kann hier wieder *D_0* mit der Sehne am Ellipsoid angenährt werden (*D_0 = s_0). Für die Reduktion in die ebene Abbildung kann ein Masstabsfaktor in Abhängigkeit der Position auf der Erde und des Erdradius berechnet werden. In der Schweiz kann dies mit dem mittleren Abstand des Polygonzugs (bei lokalen Messungen) berechnet werden. 

In [543]:
## Distances
def correction_met(d, temp, press): # watchout d in km!!! (pass as m)
    return ((-1 * (temp - T) + 0.3 * (press - P)) * (d / 1000)) / 1000 # genäherte Korrektur

def calculate_h_b(h_a, d_s, zenith, i, t):
    return h_a + d_s * np.cos(radian(zenith)) + i - t

def correction_sehne(h_a, h_b, s):
    return np.sqrt( (s**2 - np.abs(h_a - h_b)) / 
                    ( (1 + (h_a / R)) * (1 + (h_b / R)) ) 
                    )

def correction_factor_distance_flat(X_m):
    return 1 + ((X_m**2) / (2*R**2))

def average(values):
    return sum(values) / len(values)

In [544]:
## Angles
def calc_brechungswinkel(ang_0, ang_1):
    return ang_1 - ang_0

def calculate_richtungswinkel(r_0, beta):
    R = r_0 + 200 + beta

    if R < 0: R = R + 400
    elif R > 400: R = R - 400

    return R

def calculate_richtungsabschluss(coord_0, coord_1):
    return gon(np.arctan2(  (coord_0[1] - coord_1[1]), 
                        (coord_0[0] - coord_1[0]) 
                        ))

## Calculating coordinates (1. GA)
def calculate_next_coordinates(coords_stand, d, az):
    Y = coords_stand[0] + d * np.sin(radian(az))
    X = coords_stand[1] + d * np.cos(radian(az))
    return (Y, X)

## Berechnung der Distanzreduktionen und -korrekturen

Im folgenden Codeblock werden die oben definierten Funktionen angewendet. Dies ergibt einen grossen DataFrame welcher gleich darunter interpretiert wird. 

In [545]:
def calculate_corrections(df, df_met, df_fix):
    df = data_preprocessing(df, df_met, df_fix)
    temp = df_met['T'].mean()
    press = df_met['p'].mean()

    # calculate meteo correction for every leg with mean meteo meas
    df["Delta_d_met"] = df["D"].apply(correction_met, args=(temp, press)) #TODO make it use every meteo data from each Standpunkt, but it's okay as temp / press are close
    ## apply correction for distances
    df["d_m"] = df["D"] - df['Delta_d_met']

    # Lichtweg im Raum
    df["s_m"] = df["d_m"] # genähert

    # Sehne im Raum
    s_0 = list()
    for i, meas in df.iterrows():
        #TODO This is currently not working properly (h_a doesnt get updated)
        if i == 0 or i == len(data_Noise["D"]) - 1:  # ignore first and last entry
            s_0.append(pd.NA)
            continue
        df["h_b"][i] = calculate_h_b(meas["h_a"], meas["s_m"], meas["Z_m"], meas["i"], float(df_met["i"][df_met["PNr"] == meas["Visur_nach"]]))
        s_0.append(correction_sehne(meas["h_a"], df["h_b"][i], meas["s_m"]))
    df["s_0"] = s_0

    # Sehne am Ellipsoid
    df["D_0"] = df["s_0"] # genähert

    #Bogen am Ellipsoid
    X_m = df_fix["X"].mean() - 1200000 # mittlerer Abstand Bern
    correction_factor_flat = correction_factor_distance_flat(X_m)
    df["correction_factor_flat"] = correction_factor_flat
    df["D_flat"] = df["D_0"] * df["correction_factor_flat"]

    # Distanzen Mitteln
    D_m = list()
    D_m.append(pd.NA)
    for i, dist in enumerate(df["D_flat"]):
        if i == 0 or i == len(df["D_flat"]) - 1: continue # ignore first and last entry
        if i % 2 != 0:
            D_m.append((dist + df["D_flat"][i+1]) / 2)
            D_m.append((dist + df["D_flat"][i+1]) / 2)
    D_m.append(pd.NA)
    df["D_m"] = D_m

    return df

# Calculate with your preferred Dataset
data_Noise = calculate_corrections(data_Noise, i_meteo, fixpunkte)
data_Noise

data_NoNoise = calculate_corrections(data_NoNoise, i_meteo, fixpunkte)
data_NoNoise

,Visur_von,Visur_nach,RI,RII,ZI,ZII,D,i,T,p,...,h_a,h_b,Delta_d_met,d_m,s_m,s_0,D_0,correction_factor_flat,D_flat,D_m
0,A,P,165.5718,365.5718,103.4797,296.5203,NaN,1.602,18.0,959.8,...,534.139,0.000000,NaN,NaN,NaN,<NA>,<NA>,1.000033,<NA>,<NA>
1,A,PP1,365.7370,165.7370,96.7342,303.2658,27.4199,1.602,18.0,959.8,...,534.139,535.527029,-0.000596,27.420496,27.420496,27.392875,27.392875,1.000033,27.393766,27.394913
2,PP1,A,261.8157,61.8157,103.2658,296.7342,27.4199,1.620,18.0,959.6,...,1.620,0.231971,-0.000596,27.420496,27.420496,27.39517,27.39517,1.000033,27.396061,27.394913
3,PP1,PP2,60.9770,260.9770,97.2136,302.7864,25.4368,1.620,18.0,959.6,...,1.620,2.816004,-0.000553,25.437353,25.437353,25.413825,25.413825,1.000033,25.414651,25.414653
4,PP2,PP1,70.4558,270.4558,102.7864,297.2136,25.4368,1.537,17.9,959.8,...,1.537,0.340996,-0.000553,25.437353,25.437353,25.41383,25.41383,1.000033,25.414656,25.414653
5,PP2,PP3,251.6220,51.6220,98.3621,301.6379,33.4681,1.537,17.9,959.8,...,1.537,2.475993,-0.000728,33.468828,33.468828,33.454786,33.454786,1.000033,33.455874,33.455877
6,PP3,PP2,371.0396,171.0396,101.6379,298.3621,33.4681,1.459,17.7,959.8,...,1.459,0.520007,-0.000728,33.468828,33.468828,33.454792,33.454792,1.000033,33.45588,33.455877
7,PP3,PP4,151.6620,351.6620,99.3118,300.6882,23.3132,1.459,17.7,959.8,...,1.459,1.487021,-0.000507,23.313707,23.313707,23.313101,23.313101,1.000033,23.313859,23.313858
8,PP4,PP3,340.9986,140.9986,100.6882,299.3118,23.3132,1.683,17.5,959.7,...,1.683,1.654979,-0.000507,23.313707,23.313707,23.3131,23.3131,1.000033,23.313858,23.313858
9,PP4,E,168.9636,368.9636,100.4986,299.5014,44.8211,1.683,17.5,959.7,...,1.683,1.400958,-0.000975,44.822075,44.822075,44.818917,44.818917,1.000033,44.820375,44.818489


### Interpretation DataFrame
#### Distanzen
Die **Meteokorrektur** korrigiert im submillimeter Bereich (siehe *Delta_d_met*). Dies ist plausibel, da die Temperatur und der Druck doch eher einen grossen Unterschied zwischen den eingestellten Werten am Gerät und den gemessenen aufweisen. Man sollte die Meteokorrektur **auf jeden Fall durchführen**. Für eine Verbesserung im Code können die Meteodaten nicht gemittelt verwendet werden, sondern für jeden "Leg" des Zuges einzeln.
Für die Reduktion der *Sehne im Raum* zu *s_0* weist der Code einen **Fehler** bei dem Übertrag von *h_a* auf und somit auch bei der Berechnung von *h_b* (siehe entsprechende Spalten im df). Somit entsteht eine grössere Abweichung durch falsche Reduktion. 
Für die *Distanz in der ebenen Abbildung* habe ich das Mittel der Koordinaten der Fixpunkte verwendet. Es könnten auch nur Anfangs- und Endpunkt verwendet werden, aber die Fernziele E und Q sind nicht sehr weit entfernt.

### Berechnung der Koordinaten
Für die Berechnung werden folgend die Brechungswinkel im Zug, die Richtungswinkel und danach die Koordinaten mithilfe dieser Winkel und der gemittelten (reduzierten und korrigierten) Distanzen berechnet.


In [546]:
# Brechungswinkel
def brechungswinkel(df):
    beta = []
    for i, meas in df.iterrows():
        if i == len(df) - 1: continue # skip last row (already calculated)
        beta.append(calc_brechungswinkel(df["R_m"][i], df["R_m"][i + 1]))
    return pd.DataFrame(beta)

betas = brechungswinkel(data_NoNoise)
fixpunkte = fixpunkte.set_index("PNr")
betas


,0
0,0.1652
1,-103.9213
2,-0.8387
3,9.4788
4,-18.8338
5,119.4176
6,-19.3776
7,-10.6634
8,27.9650
9,-24.7581


In [547]:
# Richtungswinkel
p = (fixpunkte.loc["P", "Y"], fixpunkte.loc["P", "X"])
a = (fixpunkte.loc["A", "Y"], fixpunkte.loc["A", "X"])
e = (fixpunkte.loc["E", "Y"], fixpunkte.loc["E", "X"])
q = (fixpunkte.loc["Q", "Y"], fixpunkte.loc["Q", "X"])

t_pa = calculate_richtungsabschluss(p, a)
t_eq = calculate_richtungsabschluss(e, q)

richtungswinkel = list()
for i, beta in betas.iterrows():
    angle = calculate_richtungswinkel(t_pa, float(beta))
    richtungswinkel.append(angle)
richtungswinkel

[272.141493576587,
 168.05499357658692,
 271.13759357658694,
 281.455093576587,
 253.14249357658696,
 391.39389357658695,
 252.59869357658692,
 261.312893576587,
 299.94129357658693,
 247.21819357658694,
 289.083193576587]

In [548]:
# Richtungsabschluss / Winkelabschluss
richtungsabschluss = float(t_pa + np.sum(betas))
print("Richtungsabschluss: ", richtungsabschluss)
print("Winkelabschluss: ", float(t_eq - richtungsabschluss))

#TODO when Winkelabschlussfehler works (is small), distribute it on to the Richtungswinkel

Richtungsabschluss:  67.71689357658698
Winkelabschluss:  -1.9277277479927193


#### Interpretation Winkelabschluss
Dieser Winkelabschluss ist für einen kurzen Polygonzug deutlich zu hoch. Leider finde ich den Fehler nicht im Code.

In [549]:
# Koordinaten
coords = list()
coords.append(a) # Anfangspunkt Standpunkt

dists = data_Noise["D_m"].drop_duplicates().dropna().reset_index(drop=True) #d_m

for i, d_m in enumerate(dists):
    coords.append(calculate_next_coordinates(coords[i], d_m, richtungswinkel[i])) #append at i+1 next coords
coords = np.array(coords)

# sanity check
print(coords)
print(e)

[[2681001.605      1251457.632     ]
 [2680976.79002787 1251446.02225597]
 [2680989.01398561 1251423.74138964]
 [2680958.93936824 1251409.08850219]
 [2680936.60739092 1251402.39265954]
 [2680903.3890212  1251372.30190643]]
(2680966.888, 1251310.45)


In [550]:
# Verbesserungen
f_y, f_x = e[0] - coords[-1][0], e[1] - coords[-1][1]
print("f_y, f_x: ", f_y, f_x) #rip
#print(dists)
v_dy, v_dx = np.array(f_y * dists / np.sum(dists)).reshape(1,5), np.array(f_x * dists / np.sum(dists)).reshape(1,5)
verbesserungen = np.vstack([np.sum(v_dy), np.sum(v_dx)]).T
coords += verbesserungen

print(coords)

# sanity check after verbesserungen
f_y, f_x = e[0] - coords[-1][0], e[1] - coords[-1][1]
print("Verbesserter Abschluss: f_y, f_x: ", f_y, f_x) #yay, but that was cheating

f_y, f_x:  63.49897880433127 -61.8519064290449
[[2681065.1039788  1251395.78009357]
 [2681040.28900668 1251384.17034954]
 [2681052.51296441 1251361.88948321]
 [2681022.43834705 1251347.23659576]
 [2681000.10636972 1251340.54075312]
 [2680966.888      1251310.45      ]]
Verbesserter Abschluss: f_y, f_x:  0.0 0.0


#### Interpretation Koordinaten / Verbesserungen
Bei den Koordinaten ist etwas deutlich schief gegangen. Der Koordinatenabschluss in beide Richtungen (*f_y* / *f_x*) ist zu gross. Die Verbesserung gleicht den Polygonzug auf Anfangs- und Endpunkt aus (siehe *Verbesserter Abschlusss*). Man kann aber die gemessenen Neupunkte in dieser Form nicht verwenden. 

## Fazit
Es ist immens wichtig, Plausibilitätschecks durchzuführen, um Fehler bei Messungen und beim Postprocessing aufzudecken. Auch wurde zu viel Zeit in die Generalisierung der Funktionen im ersten Teil des Codes gesteckt. So blieb zum Schluss nur wenig Zeit, Implementationsfehler auszumerzen (und einige wurden nicht behoben). 